In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
from selenium.webdriver.common.by import By
import requests
import ast
import plotly.graph_objects as go
import plotly.express as px
from tqdm import tqdm
from datetime import datetime

In [2]:

df_datasets = pd.read_excel('./src/combinaciones_palabras.xlsx')


In [3]:

#df_datasets = df_datasets[df_datasets['Words'].str.contains('Analytical modeling|Food insecurity')]
#df_datasets.reset_index(drop=True, inplace=True)
#df_datasets

In [4]:
driver = webdriver.Chrome()
# Abre la página web
campo = 'auxiliar de contabilidad'
url = 'https://grants.nih.gov/funding/searchguide/index.html#/'
driver.get(url)

In [5]:

def find_number_results(word):

    input_element = driver.find_element(By.ID, "searchInputBox")
    input_element.clear()
    

    input_element.send_keys(f'"{word}"')
    time.sleep(7)
    driver.find_element(By.CLASS_NAME, "input-group-text.primary").click()

    time.sleep(7)

    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        span_tag = soup.find('span', attrs={'ng-show': 'totalresults > 0'}).text
        expresion_regular = r"Displaying: \d+ to \d+ of (\d+) results"
        resultado = re.search(expresion_regular, span_tag)
        valor_total = int(resultado.group(1))
        return word, valor_total
    except:
        return word, 0


In [6]:
results = []

list = {}
total_words = len(df_datasets)  

select_element = driver.find_element(By.ID, 'inlineType')
select = Select(select_element)
select.select_by_value('object:7')

for i in tqdm(range(total_words), desc="Procesando palabras"):
    #print(df_datasets['Words'][i])
    temporal = find_number_results(df_datasets['Words'][i])
    results.append([temporal[0], temporal[1]])

    if int(temporal[1]) > 0 and  int(temporal[1])<100:
        select_element = driver.find_element(By.ID, "selectPerPage")
        select = Select(select_element)
        select.select_by_value("number:100")

        
        soup = ''
        #driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)
        soup = BeautifulSoup (driver.page_source, 'html.parser')
            
        for row in soup.select('tr.ng-scope'):
            funding = {}
            url = row.select('td a')[0]['href']
            #print(url)
            funding['word'] = df_datasets['Words'][i]
            funding['title'] = row.select('td')[0].text.strip()
            funding['nofo'] = row.select('td')[1].text.strip()
            funding['issuing_organitation'] = row.select('td')[2].text.strip()

            release_date = row.select('td')[3].text.strip()
            if release_date != 'N/A':
                funding['release_date'] = datetime.strptime(release_date, '%b %d, %Y')
            else:
                funding['release_date'] = release_date

            expiration = row.select('td')[4].text.strip()
            if expiration != 'N/A':
                funding['expiration'] = datetime.strptime(expiration, '%b %d, %Y')
            else:
                funding['expiration'] = expiration

            funding['activity_code'] = row.select('td')[5].text.strip()
            list[url] = funding

driver.quit()

df = pd.DataFrame.from_dict(list, orient='index')
df['url'] = df.index

df = df.reset_index(drop=True)
df.head()
df.to_excel('./src/listado.xlsx')


df_resultado = pd.DataFrame(results, columns=['words', 'Results'])

print("Proceso completado. Resultados almacenados en df_resultado.")
df_resultado.to_excel('./src/Resultados.xlsx')




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="inlineType"]"}
  (Session info: chrome=129.0.6668.90); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60978B675+29573]
	(No symbol) [0x00007FF6097004A0]
	(No symbol) [0x00007FF6095BB6EA]
	(No symbol) [0x00007FF60960F815]
	(No symbol) [0x00007FF60960FA6C]
	(No symbol) [0x00007FF60965B917]
	(No symbol) [0x00007FF60963733F]
	(No symbol) [0x00007FF6096586BC]
	(No symbol) [0x00007FF6096370A3]
	(No symbol) [0x00007FF6096012DF]
	(No symbol) [0x00007FF609602441]
	GetHandleVerifier [0x00007FF609ABC5BD+3375821]
	GetHandleVerifier [0x00007FF609B079B7+3684039]
	GetHandleVerifier [0x00007FF609AFCDDB+3640043]
	GetHandleVerifier [0x00007FF60984B7F6+816390]
	(No symbol) [0x00007FF60970B7AF]
	(No symbol) [0x00007FF6097075D4]
	(No symbol) [0x00007FF60

#################################################################

Segunda Parte

#################################################################

### Recorrido por cada una de las URLS

In [5]:
pd_listado_url = pd.read_excel('./listado.xlsx')

In [60]:
for i in range(len(pd_listado_url.head(10))):

    print(pd_listado_url['url'][i], ' - ', pd_listado_url['title'][i])
    
    response = requests.get(pd_listado_url['url'][i])
    html_content = response.text

    html = BeautifulSoup (html_content, 'html.parser')

    nombre_archivo = str('./funding/{}.txt').format(str(pd_listado_url['url'][i]).split('/')[-1:][0].replace('.html',''))
    with open(nombre_archivo, 'w') as archivo:
        archivo.write(str(html))
    time.sleep(3)
    

https://grants.nih.gov/grants/guide/notice-files/NOT-OD-22-135.html  -  Notice of Special Interest (NOSI): Stimulating Research to Understand and Address Hunger, Food and Nutrition Insecurity


Notice of Special Interest (NOSI): Stimulating Research to Understand and Address Hunger, Food and Nutrition Insecurity
https://grants.nih.gov/grants/guide/rfa-files/RFA-MD-24-005.html  -  Elucidating Mechanisms Associated with HIV Related Co-Morbidities in Populations Experiencing Health Disparities (R01 - Clinical Trials Not Allowed)
https://grants.nih.gov/grants/guide/pa-files/PAS-24-163.html  -  Priority HIV/AIDS Research within the Mission of NIDDK (R01 Clinical Trial Optional)
https://grants.nih.gov/grants/guide/rfa-files/RFA-DK-25-001.html  -  Addressing the Impact of Syndemics on the Health of People with HIV and Diseases and Conditions within the Missions of NIDDK and NHLBI (R01 Clinical Trial Optional)
https://grants.nih.gov/grants/guide/rfa-files/RFA-DA-25-048.html  -  Seeking Product